In [1]:
import wandb
api = wandb.Api()


In [2]:
group_name = "Sweep delta, seeds, and omit_states_actions with no walls and Q value output (run for longer)."

In [9]:
import pandas as pd
runs = api.runs("rldl/icvi", {"$and": [{"group": group_name}]})

def generate_dfs():
    for run in runs:
        history = pd.DataFrame(run.history())
        history["seed"] = run.config["seed"]
        history["omit_states_actions"] = run.config["train_data_args"]["omit_states_actions"]
        history["delta"] = run.config["bellman_delta"]
        yield history

orig = pd.concat(list(generate_dfs()))
orig.columns

Index(['fps', 'train-n/mae', 'test-1/loss', 'test-n/values-plot 44',
       'test-n/values-plot 97', 'epoch', 'train-n/rmse',
       'test-n/values-plot 36', 'train-1/n', 'test-1/rmse', 'train-1/rmse',
       'test-n/mae', '_timestamp', '_runtime', 'test-1/pair_wise_accuracy',
       'test-1/improved_policy_value', 'train-1/mae', 'test-1/mae',
       'train-1/loss', 'train-1/pair_wise_accuracy', 'lr', 'test-n/regret',
       'test-1/values-plot 97', '_step', 'save_count', 'test-1/regret',
       'test-n/loss', 'test-n/improved_policy_value', 'test-1/values-plot 36',
       'test-n/rmse', 'test-1/values-plot 44', 'test-n/pair_wise_accuracy',
       'train-n/pair_wise_accuracy', 'seed', 'omit_states_actions', 'delta',
       'test-n/values-plot 168', 'test-1/values-plot 168',
       'test-1/values-plot 65', 'test-1/values-plot 161',
       'test-n/values-plot 65', 'test-n/values-plot 161', 'train-n/regret',
       'train-n/improved_policy_value'],
      dtype='object')

In [21]:
grouped = orig.groupby('_step')['test-n/rmse'].agg(['mean', 'std'])
grouped['test_upper'] = (grouped['mean'] + grouped['std']).clip(lower=1e6)
grouped['test_lower'] = (grouped['mean'] - grouped['std']).clip(lower=1e6)
merged = orig.merge(grouped[['test_upper', 'test_lower']], on='_step', how='left')


In [22]:
grouped = orig.groupby('_step')['test-n/rmse'].agg(['mean', 'std'])
grouped['train_upper'] = (grouped['mean'] + grouped['std']).clip(lower=1e6)
grouped['train_lower'] = (grouped['mean'] - grouped['std']).clip(lower=1e6)
merged = orig.merge(grouped[['train_upper', 'train_upper']], on='_step', how='left')

In [23]:
renamed = merged.rename(columns={"_step": "step", "test-n/rmse": "test", "train-n/rmse": "train"})

In [24]:
import itertools
for (delta, omit_states_actions) in itertools.product(renamed.delta.unique(), renamed.omit_states_actions.unique()):
    data = renamed[(renamed["delta"] == delta) & (renamed["omit_states_actions"] == omit_states_actions)]
    data = data.reset_index()
    data.to_json(f"data/delta_{delta}_omit_states_actions_{omit_states_actions}.json", orient="records")